In [114]:
import numpy as np

In [115]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화)
    (N,C,H,W) => (N*OH*OW, C*FH*FW)
    """

    # N: 배치 수, C: 채널 수, H: 입력 이미지 높이, W: 입력 이미지 너비
    N, C, H, W = input_data.shape

    # 입력데이터와 필터의 합성곱 연산 결과의 크기를 미리 계산한다.
    out_h = (H + 2*pad - filter_h) // stride + 1 # //는 정수 나눗셈 (몫 연산) 이다. (소수점 이하를 버림)
    out_w = (W + 2*pad - filter_w) // stride + 1

    # 입력데이터에 패딩을 적용한다.
    # N차원과 C차원에는 패딩을 적용하지 않고, H와 W차원에만 pad값으로 패딩한다.
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')

    # 0: N - 입력 데이터의 수
    # 1: C - 채널 수
    # 2: Filter Height
    # 3: Filter Width
    # 4: Output Height
    # 5: Output Width
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    # 필터의 높이와 너비에 따라 슬라이딩하며 이미지를 분할한다.
    for y in range(filter_h):
        y_max = y + stride*out_h

        for x in range(filter_w):
            x_max = x + stride*out_w

            # 슬라이싱을 사용하여 x ~ x_max 범위 안에서 스트라이드 간격으로 데이터를 추출한다.
            # 필터가 상위 차원이며, 윈도우가 움직였을 때의 값들을 그 하위 차원으로 저장한다(out_h, out_w)
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    # N,C,FH,FW,OH,OW => N,OH,OW,C,FH,FW
    # 1) 필터의 차원이 상위에 있던 배열을 슬라이딩 차원(OH, OW)을 상위 차원으로 배치하여 변환한다.
    # 2) 행을 배치 수(N) * 슬라이딩 수(OH*OW)로 하고 나머지 요소(C*FH*FW)를 열로하는 2차원 배열로 변환한다.
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)

    return col

x1 = np.random.rand(1, 1, 5, 5)
col1 = im2col(x1, filter_h=3, filter_w=3, stride=2, pad=0)
print(x1.shape)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(x2.shape)
print(col2.shape)

(1, 1, 5, 5)
(4, 9)
(10, 3, 7, 7)
(90, 75)


In [116]:
class ConvolutionLayer:
    """합성곱 계층
    """
    
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W # (FN, C, FH, FW)
        self.b = b # (FN, )
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        OH = (H + 2*self.pad - FH) // self.stride + 1
        OW = (W + 2*self.pad - FW) // self.stride + 1

        col = im2col(x, FH, FW, self.stride, self.pad) # (N,OH,OW,C,FH,FW) => (N*OH*OW, C*FH*FW)
        col_W = self.W.reshape(FN, -1).T # (FN, C, FH, FW) => (FN, C*FH*FW) => (C*FH*FW, FN)
        out = np.dot(col, col_W) + self.b # (N*OH*OW, C*FH*FW) dot (C*FH*FW, FN) => (N*OH*OW, FN)
        out = out.reshape(N, OH, OW, -1) # (N, OH, OW, FN)
        out = out.transpose(0, 3, 1, 2) # (N, OH, OW, FN) => (N, FN, OH, OW)
        return out

In [ ]:
class PoolingLayer:
    """풀링 계층
    """

    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        OH = (H + 2*self.pad - self.pool_h) // self.stride + 1
        OW = (W + 2*self.pad - self.pool_w) // self.stride + 1

        # (N, C, H, W) => (N, OH, OW, C, PH, PW) => (N*OH*OW, C*PH*PW)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)

        # (N*OH*OW, C*PH*PW) => (N*OH*OW*C, PH*PW)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        # 열 데이터 중 최대값만 추출하여 행으로 모음.
        # (N*OH*OW*C, max(PH*PW))
        out = np.max(col, axis=1)

        # (N*OH*OW*C, max(PH*PW)) => (N, OH, OW, C) => (N, C, OH, OW)
        out = out.reshape(N, OH, OW, C).transpose(0, 3, 1, 2)

        return out